# Vehicle Routing Problem

Solving the vehicle routing problem using OR-tools by google. Toy problem to explore optimization tools

In [80]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp


First, I create a distance matrix using an API key

In [81]:
import os
from dotenv import load_dotenv

load_dotenv("keys.env")

api_key = os.getenv('DISTANCE_MATRIX')

In [82]:
import requests
import json
import urllib


def create_data():
  """Creates the data."""
  data = {}
  data['API_key'] = api_key
  data['addresses'] = ['7426+Hubbard+Avenue+Middleon+WI+53562', # depot
                       '2100+Bristol+St+Middleton+WI+53562', # high school
                       '2616+N+Pleasant+View+Rd+Middleton+WI+53562', # ice rink
                       '3650+University+Ave+Shorewood+Hills+WI+53705', # grocery store
                       '1401+Observatory+Dr+Madison+WI+53706', # UW observatory
                      ]
  return data

def create_distance_matrix(data):
  addresses = data["addresses"]
  API_key = data["API_key"]
  # Distance Matrix API only accepts 100 elements per request, so get rows in multiple requests.
  max_elements = 100
  num_addresses = len(addresses) # 16 in this example.
  # Maximum number of rows that can be computed per request (6 in this example).
  max_rows = max_elements // num_addresses
  # num_addresses = q * max_rows + r (q = 2 and r = 4 in this example).
  q, r = divmod(num_addresses, max_rows)
  dest_addresses = addresses
  distance_matrix = []
  # Send q requests, returning max_rows rows per request.
  for i in range(q):
    origin_addresses = addresses[i * max_rows: (i + 1) * max_rows]
    response = send_request(origin_addresses, dest_addresses, API_key)
    distance_matrix += build_distance_matrix(response)

  # Get the remaining remaining r rows, if necessary.
  if r > 0:
    origin_addresses = addresses[q * max_rows: q * max_rows + r]
    response = send_request(origin_addresses, dest_addresses, API_key)
    distance_matrix += build_distance_matrix(response)
  return distance_matrix

def send_request(origin_addresses, dest_addresses, API_key):
  """ Build and send request for the given origin and destination addresses."""
  def build_address_str(addresses):
    # Build a pipe-separated string of addresses
    address_str = ''
    for i in range(len(addresses) - 1):
      address_str += addresses[i] + '|'
    address_str += addresses[-1]
    return address_str

  request = 'https://maps.googleapis.com/maps/api/distancematrix/json?units=metric'
  origin_address_str = build_address_str(origin_addresses)
  dest_address_str = build_address_str(dest_addresses)
  request = request + '&origins=' + origin_address_str + '&destinations=' + \
                       dest_address_str + '&key=' + API_key
  jsonResult = urllib.request.urlopen(request).read()
  response = json.loads(jsonResult)
  return response

def build_distance_matrix(response):
  distance_matrix = []
  for row in response['rows']:
    row_list = [row['elements'][j]['distance']['value'] for j in range(len(row['elements']))]
    distance_matrix.append(row_list)
  return distance_matrix

# Create the data.
data = create_data()
addresses = data['addresses']
API_key = data['API_key']
distance_matrix = create_distance_matrix(data)
print(distance_matrix)

[[0, 1115, 3566, 6308, 9852], [995, 0, 3796, 6451, 9995], [3479, 4560, 0, 9213, 12757], [6082, 6219, 9067, 0, 4040], [10237, 10373, 13222, 4309, 0]]


With this distance matrix, we can now create the data that will be passed to the optimizer

In [83]:
def create_data_model(distance_matrix, num_vehicles, depot):
    """Store the data for the problem"""
    data = {}
    data["distance_matrix"] = distance_matrix
    data["num_vehicles"] = num_vehicles
    data["depot"] = 0

    return data

data = create_data_model(distance_matrix, 2, 0)

Create the index manager and the routing model.

In [84]:
manager = pywrapcp.RoutingIndexManager(len(data["distance_matrix"]), data["num_vehicles"], data["depot"]) # inputs: number of locations, number of vehicles, starting point

routing = pywrapcp.RoutingModel(manager)

Create a function that takes any pair of locations and returns the distance between them

In [85]:
def distance_callback(from_index, to_index):
    """Returns the distance between the two nodes."""
    # Convert from routing variable Index to distance matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data["distance_matrix"][from_node][to_node]

transit_callback_index = routing.RegisterTransitCallback(distance_callback)


Now set the cost of travel, which tells the solver how to caluclate the cost of travel between any two locations

In [86]:
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

Create a distance dimension, which computes the cumulative distance traveled by each vehicle. The total cost can be made proportional to the maximum of th etotla distances along each route. This ensures that no one vechicle travels much longer than others.

In [87]:
dimension_name = "Distance"
routing.AddDimension(
    transit_callback_index,
    0,  # no slack
    36000,  # vehicle maximum travel distance
    True,  # start cumul to zero
    dimension_name,
)
distance_dimension = routing.GetDimensionOrDie(dimension_name)
distance_dimension.SetGlobalSpanCostCoefficient(100)


Find a solution. This sets the solution strategy to PATH_CHEAPEST_ARC.

In [88]:
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
)

The following function prints the solution returned by the solver

In [93]:
indices_to_names = {0:"Depot", 1:"School", 2:"Rink", 3:"Grocery", 4:"Observatory"}
def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f"Objective: {solution.ObjectiveValue()}")
    max_route_distance = 0
    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        plan_output = f"Route for vehicle {vehicle_id}:\n"
        route_distance = 0
        while not routing.IsEnd(index):

            place = indices_to_names[manager.IndexToNode(index)]

            plan_output += f" {place} -> "
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id
            )
        plan_output += f"{indices_to_names[manager.IndexToNode(index)]}\n"
        plan_output += f"Distance of the route: {route_distance}m\n"
        print(plan_output)
        max_route_distance = max(route_distance, max_route_distance)
    print(f"Maximum of the route distances: {max_route_distance}m")


Run and solve!

In [94]:
# Solve the problem.
solution = routing.SolveWithParameters(search_parameters)
# Print solution on console.
if solution:
    print_solution(data, manager, routing, solution)
else:
    print("No solution found !")


Objective: 2048791
Route for vehicle 0:
 Depot ->  Grocery ->  School ->  Rink -> Depot
Distance of the route: 19802m

Route for vehicle 1:
 Depot ->  Observatory -> Depot
Distance of the route: 20089m

Maximum of the route distances: 20089m
